In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import hockey.hockey_env as h_env

from rl_hockey.sac import SAC
from rl_hockey.common import utils

In [26]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)

o_space = env.observation_space
ac_space = env.action_space

In [21]:
agent = SAC(o_space.shape[0], action_dim=ac_space.shape[0] // 2, noise='pink', max_episode_steps=500)
agent.load("../../../results/hyperparameter_runs/2026-01-17_20-35-18/models/run_lr1e03_bs256_h128_128_128_4c1f51eb_20260117_203518_vec16.pt")

In [22]:
opponent = h_env.BasicOpponent(weak=False)

In [23]:
N = 100
win_count = 0
total_reward = 0

In [27]:
for i in tqdm(range(N)):
    state, _ = env.reset()
    for t in range(500):
        env.render(mode="human")

        done = False
        action1 = agent.act(state.astype(np.float32), deterministic=True)
        action2 = opponent.act(env.obs_agent_two())

        (next_state, reward, done, trunc, info) = env.step(np.hstack([action1, action2]))
        state = next_state

        total_reward += reward

        if done or trunc:
            break
    
    if info['winner'] == 1:
        win_count += 1

print(f"Average Reward over {N} episodes: {total_reward / N}")
print(f"Win Rate over {N} episodes: {win_count / N}")

 46%|████▌     | 46/100 [00:48<00:57,  1.06s/it]


KeyboardInterrupt: 

In [28]:
env.close()